In [1]:
#Python
#import module for predict
import os
import numpy as np
import configparser
import cv2
from matplotlib import pyplot as plt
#Keras
from keras.models import model_from_json
from keras.models import Model
#scikit learn
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import f1_score
import sys
sys.path.insert(0, './lib_keras/')

# help_functions.py
from help_functions import *
# extract_patches.py
from temp_extract_patches import recompose_img
from temp_extract_patches import recompose_overlap_img
from temp_extract_patches import paint_border
from temp_extract_patches import kill_border
from temp_extract_patches import pred_only_FOV
from temp_extract_patches import get_data_testing
from temp_extract_patches import get_data_testing_overlap
# pre_processing.py
from pre_processing import my_preprocessing



Using TensorFlow backend.
/home/bono/.pyenv/versions/3.5.5/envs/gpuTest/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/bono/.pyenv/versions/3.5.5/envs/gpuTest/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/bono/.pyenv/versions/3.5.5/envs/gpuTest/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np

In [2]:
#========= CONFIG FILE TO READ FROM =======
config = configparser.RawConfigParser()
config.read('configuration.txt')

#run the training on invariant or local
path_data = config.get('data paths', 'path_local')
# original test images (for FOV selection)
# init setting (PATH, mode, parameter)
mode = config.get('testing settings', 'mode')

if mode == 'over_train':
    OCT_test_ori_path = path_data + config.get('data paths', 'train_imgs_original')

elif mode == 'test':
    OCT_test_ori_path = path_data + config.get('data paths', 'test_imgs_original')

# dimension of the patches
patch_height = int(config.get('data attributes', 'patch_height'))
patch_width = int(config.get('data attributes', 'patch_width'))
#the stride in case output with average
stride_height = int(config.get('testing settings', 'stride_height'))
stride_width = int(config.get('testing settings', 'stride_width'))
assert (stride_height < patch_height and stride_width < patch_width)
#model name
name_experiment = config.get('experiment name', 'name')
target_train = config.get('experiment name', 'result_save_path')

path_experiment = './'+'result/'+ name_experiment
#N full images to be predicted
Imgs_to_test = int(config.get('testing settings', 'full_images_to_test'))
#Grouping of the predicted images
N_visual = int(config.get('testing settings', 'num_group_visual'))
#====== average mode ===========
average_mode = config.getboolean('testing settings', 'average_mode')

print(name_experiment)
print(target_train)


20_0621_fine_segmentor_180_Data_focal_gamma_07
result


In [3]:
test_img_ori = load_hdf5(OCT_test_ori_path)
full_img_height = test_img_ori.shape[2]
full_img_width = test_img_ori.shape[3]
print(np.shape(test_img_ori))

In [4]:
#============ Load the data and divide in patches
patches_imgs_test = None
new_height = None
new_width = None
masks_test  = None
patches_masks_test = None
if average_mode == True:
    patches_imgs_test, new_height, new_width = get_data_testing_overlap(
        DRIVE_test_img_ori_path = OCT_test_ori_path,
        num_test_img = int(config.get('testing settings', 'full_images_to_test')),
        patch_h = patch_height,
        patch_w = patch_width,
        stride_h = stride_height,
        stride_w = stride_width
    )
else:
    patches_imgs_test = get_data_testing(
        test_img_ori_path = OCT_test_ori_path,  #original
        num_test_img = int(config.get('testing settings', 'full_images_to_test')),
        patch_h = patch_height,
        patch_w = patch_width
    )

test img shape :  (176, 3, 64, 64)

[get_data_testing_fucn] test PATCHES images/grds shape:
(176, 1, 64, 64)
[get_data_testing_fucn] test PATCHES images range (min-max): 0.0 - 1.0


In [5]:
#================ Run the prediction of the patches ==================================
best_last = config.get('testing settings', 'best_last')
#Load the saved model
model = model_from_json(open(path_experiment+'/'+ name_experiment +'_architecture.json').read())
model.load_weights(path_experiment+'/'+best_last+'_weights.h5')
#Calculate the predictions
predictions = model.predict(patches_imgs_test, batch_size=64, verbose=2)
print ("predicted images size :",predictions.shape)


predicted images size : (176, 4096, 3)


In [6]:
#===== Convert the prediction arrays in corresponding images
pred_patches = pred_to_imgs(predictions, patch_height, patch_width, "original")
pred_patches_thr = pred_to_imgs(predictions, patch_height, patch_width, "threshold")

pred imgs shape :  (176, 4096, 2)
pred imgs shape :  (176, 4096, 2)


In [7]:
pred_imgs = pred_patches
pred_imgs_thr = pred_patches_thr

In [8]:
print(np.shape(pred_imgs))


(176, 2, 64, 64)


In [9]:


# original
pred_imgs_class01 = pred_imgs[:,0]
pred_imgs_class01 = np.expand_dims(pred_imgs_class01, 1)
pred_imgs_class02 = pred_imgs[:,1]
pred_imgs_class02 = np.expand_dims(pred_imgs_class02, 1)


# thr mode

pred_thr_imgs_class01 = pred_imgs_thr[:,0]
pred_thr_imgs_class01 = np.expand_dims(pred_thr_imgs_class01, 1)
pred_thr_imgs_class02 = pred_imgs_thr[:,1]
pred_thr_imgs_class02 = np.expand_dims(pred_thr_imgs_class02, 1)


visualize(group_images(pred_imgs_class01,N_visual),path_experiment+'/'+"all_predictions_class01")#.show()
visualize(group_images(pred_imgs_class02,N_visual),path_experiment+'/'+"all_predictions_class02")#.show()


visualize(group_images(pred_thr_imgs_class01,N_visual),path_experiment+'/'+"all_thr_predictions_class01")#.show()
visualize(group_images(pred_thr_imgs_class02,N_visual),path_experiment+'/'+"all_thr_predictions_class02")#.show()


#visualize results comparing mask and prediction:

#assert (orig_imgs.shape[0]==pred_imgs.shape[0] and orig_imgs.shape[0]==gtruth_masks.shape[0])
N_predicted = 20
group = N_visual
assert (N_predicted%group==0)
print('N_visual : ', group)


[group images func] prev data shape  : (176, 1, 64, 64)
[group images func] after data shape :  (176, 64, 64, 1)
[group images func] first total image :  (64, 64, 1)
[group images func] final total image :  (11328, 64, 1)
data shape :  (11328, 64, 1)
<PIL.Image.Image image mode=L size=64x11328 at 0x7EFF8030EA58>
file name :  ./result/20_0410_fine_segmentor_newData07/all_predictions_class01
[group images func] prev data shape  : (176, 1, 64, 64)
[group images func] after data shape :  (176, 64, 64, 1)
[group images func] first total image :  (64, 64, 1)
[group images func] final total image :  (11328, 64, 1)
data shape :  (11328, 64, 1)
<PIL.Image.Image image mode=L size=64x11328 at 0x7EFF802E95C0>
file name :  ./result/20_0410_fine_segmentor_newData07/all_predictions_class02
[group images func] prev data shape  : (176, 1, 64, 64)
[group images func] after data shape :  (176, 64, 64, 1)
[group images func] first total image :  (64, 64, 1)
[group images func] final total image :  (11328,

In [10]:
np.max(pred_imgs_class01)

0.7415329813957214

In [11]:
result_path = 'each_results'
if os.path.isdir(path_experiment+'/'+result_path) == False:
    os.mkdir(path_experiment+'/' +result_path)
else:
    print('already exist the folder in this path : {}'.format(path_experiment +result_path))

for i in range(int(N_predicted/group)):
    
    pred_ori_class01 = group_images(pred_imgs_class01[i*group:(i*group)+group,:,:,:],group)
    pred_ori_class02 = group_images(pred_imgs_class02[i*group:(i*group)+group,:,:,:],group)

    pred_stripe_class01 = group_images(pred_thr_imgs_class01[i*group:(i*group)+group,:,:,:],group)
    pred_stripe_class02 = group_images(pred_thr_imgs_class02[i*group:(i*group)+group,:,:,:],group)
 
    #total_img = np.concatenate((orig_stripe,pred_stripe_class01,pred_stripe_class02),axis=0)
    
    visualize(pred_ori_class01,path_experiment+'/'+ result_path +'/'+"Prediction_class01"+str(i))#.show()
    visualize(pred_ori_class02,path_experiment+'/'+ result_path +'/'+"Prediction_class02"+str(i))#.show()
  
    visualize(pred_stripe_class01,path_experiment+'/'+ result_path +'/'+"thr_Prediction_class01"+str(i))#.show()
    visualize(pred_stripe_class02,path_experiment+'/'+ result_path +'/'+"thr_Prediction_class02"+str(i))#.show()
    
def decoding(labels,class_num):
    '''
    input : patchs (ch first images)
            (ch, H, W)
    
    mapping example : 
        mapping = {
            (0,0,0) : 0,
            (255,0,0) : 1,
            (0,0,255) : 2
            }
    
    return patches (num, class-1,H, W)
    '''
    #print('[ch-wise one hot encoding] label shape : ', np.shape(labels))
    c, h, w = labels.shape
    decoded_labels = np.zeros((h,w,3)) # RGB format
    
    ch_cnt =0 
    temp_label = np.transpose(labels, (1,2,0))
    #print(np.shape(temp_label))

    '''
    mapping 순서 이슈
    '''
    #print('[ch wise one hot] shape label : ', np.shape(temp_label))
    #print('[ch wise one hot] shape encoded label : ',np.shape(encoded_labels))
    for k in range(class_num):
        '''
        temp label
            (H,W)
        '''
        
        row,col = np.nonzero(temp_label[:,:,k])
        print(len(row))
        if k==0:
            class_01_info = (row, col)
        elif k==1:
            class_02_info = (row, col)

            
    decoded_labels[class_01_info] = (255,0,0)
    decoded_labels[class_02_info] = (255,0,0)
    
    return decoded_labels

for i in range(pred_imgs.shape[0]):
    cv2.imwrite(path_experiment+'/'+ result_path+'/'+'pred_thr_'+str(i)+'_.png', decoding(pred_imgs_thr[i],pred_imgs_thr.shape[1]))
    #cv2.imwrite(path_experiment+'/'+ result_path+'/'+'pred_'+str(i)+'_.png', pred_imgs[i],pred_imgs_thr.shape[1])


already exist the folder in this path : ./result/20_0410_fine_segmentor_newData07each_results
[group images func] prev data shape  : (1, 1, 64, 64)
[group images func] after data shape :  (1, 64, 64, 1)
[group images func] first total image :  (64, 64, 1)
[group images func] final total image :  (128, 64, 1)
[group images func] prev data shape  : (1, 1, 64, 64)
[group images func] after data shape :  (1, 64, 64, 1)
[group images func] first total image :  (64, 64, 1)
[group images func] final total image :  (128, 64, 1)
[group images func] prev data shape  : (1, 1, 64, 64)
[group images func] after data shape :  (1, 64, 64, 1)
[group images func] first total image :  (64, 64, 1)
[group images func] final total image :  (128, 64, 1)
[group images func] prev data shape  : (1, 1, 64, 64)
[group images func] after data shape :  (1, 64, 64, 1)
[group images func] first total image :  (64, 64, 1)
[group images func] final total image :  (128, 64, 1)
data shape :  (128, 64, 1)
<PIL.Image.Imag